In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.io as pio
import functions as fun
import importlib as imp

# Set additional settings for warnings and templates
import warnings
warnings.filterwarnings("ignore")
pio.templates.default = "plotly_white"
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{: .3f}".format

In [24]:
# Reload functions if they were changed
imp.reload(fun)

<module 'functions' from 'c:\\Users\\user\\OneDrive\\PD\\ММСС\\early_warning_model\\functions.py'>

In [4]:
# Params for all simulations
number = 1000

### Simulation of the random time series

In [45]:
# Params for random simulations
length = 5000
mean = 0
sigma = 100

In [53]:
# Create simulations
for type in ['normal', 'rw']:
    data = fun.generate_random_series(length, number, mean, sigma, type)
    data.to_parquet(f'Definition_simulations/original/{type}.parquet')

### Simulation of the time series by sand piles

In [35]:
# Params for sand pile simulations
number_of_nodes = 500
length = number_of_nodes * 5
ER_prob = 4 / number_of_nodes
CL_average = 12
dist = 'uni'

In [38]:
# Simulate sand pile for 4 combinations of types and models
for type in ['ER', 'CL']:
    for model in ['BTW', 'MA']:
        data = pd.DataFrame()
        for i in tqdm(range(number)):
            G = fun.graph_generation(graph_type = type, number_of_nodes = number_of_nodes, ER_prob = ER_prob, CL_average = CL_average)
            grains = fun.grain_generator(number_of_nodes, length, dist)
            data[str(i)] = fun.spread(model, G, length, grains)
        data.to_parquet(f'Definition_simulations/original/{type}_{model}.parquet')

100%|██████████| 1000/1000 [50:43<00:00,  3.04s/it]


### Choice of the best border to define critical transition

In [28]:
crit = np.linspace(1.1, 1.5, 9)
# window = np.linspace(50, 300, 6)
window = [100]
stats = pd.DataFrame(columns = ['Sim', 'Crit', 'Window', 'Mean'])

# Load simulated time series
for i in tqdm([i for i in os.listdir('Definition_simulations/original') if i.endswith('.parquet')]):
    for c in crit:
        for w in window:
            # Calculate the critical transition
            data = fun.critical_transition(pd.read_parquet(f'Definition_simulations/original/{i}'), c, w)
            data_rise = data[data.columns[data.columns.str.contains(', Rise')]]
            data_rise.to_parquet('Definition_simulations/stats/' + i.split('.')[0] + '_' + str(round(c, 2)).replace('.', '_') + '_' + str(w) + '.parquet')
            stats = stats.append({'Sim': i.split('.')[0], 'Crit': c, 'Window': w, 
                                  'Mean': data_rise.values.sum() / data_rise.shape[1]}, ignore_index = True)
stats.to_parquet('Definition_simulations/stats/stats.parquet')
stats

100%|██████████| 6/6 [04:38<00:00, 46.45s/it]


,Sim,Crit,Window,Mean
0,CL_BTW,1.100,100,146.670
1,CL_BTW,1.150,100,68.724
2,CL_BTW,1.200,100,31.487
3,CL_BTW,1.250,100,16.328
4,CL_BTW,1.300,100,9.560
5,CL_BTW,1.350,100,5.830
6,CL_BTW,1.400,100,3.699
7,CL_BTW,1.450,100,2.411
8,CL_BTW,1.500,100,1.545
9,CL_MA,1.100,100,110.466
